In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression

# Load your data
df = pd.read_csv('duke_gpa.csv')

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['gpa', 'studyweek', 'sleepnight', 'out']),
        ('cat', OneHotEncoder(), ['gender'])
    ])

# Define the feature selection
selector = SelectKBest(f_classif, k=5)

# Define the classifier
classifier = LogisticRegression()

# Combine preprocessing, feature selection and classification in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('selector', selector),
                           ('classifier', classifier)])

# Split your data into features (X) and target (y)
X = df.drop('studyweek', axis=1)
y = df['gpa']

# Fit the pipeline to the data
pipeline.fit(X, y)

# You can now use the 'pipeline' object to make predictions

ValueError: A given column is not a column of the dataframe